# Introduction

Takes exploratory data and fits variations of the DDM.

DDM: average-signal

Details: Take every signal that the subject saw in a trial, and average their values. Feed that into a DDM as a time-invariant signal.

## Preamble

In [ ]:
# Install (package verification, PyDDM, timer, parallelization)
#!pip install paranoid-scientist
#!pip install pyddm
#!pip install pytictoc  
#!pip install pathos  

In [11]:
# Libraries
import os
from pytictoc import TicToc
import csv
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import pyddm as ddm
from pyddm import Model, Sample, FitResult, Fittable, Fitted, ICPoint, set_N_cpus
from pyddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayUniformMixture, LossRobustBIC
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

In [12]:
# Directories
datadir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-common-consequence\\data\\processed_data"
ddmdir = "D:\\OneDrive - California Institute of Technology\\PhD\\Rangel Lab\\2023-common-consequence\\analysis\\outputs\\ddm"

In [13]:
# Parallel settings
ncpu = multiprocessing.cpu_count()-1 # always save one core
ncpu

11

## Import and Clean Raw Data

rawdata_in: odd trials used for in-sample data for model fitting.

rawdata_out: even trials used for out-sample data for model predictions.

In [21]:
# Import
rawdata = pd.read_csv(datadir+"\\pilotdata.csv")

# Rename variables
data = rawdata.rename(columns={
    "Participant.Private.ID":"subject", 
    "Trial.Number":"trial",
    "Reaction.Time":"rt",
    "Response":"choice",
})

# RT in s.
data.rt = data.rt/1000

# Expected value of left vs right.
data['vL'] = data.L1H*data.L1PrH + data.L1M*data.L1PrM + data.L1L*data.L1PrL
data['vR'] = data.L2H*data.L2PrH + data.L2M*data.L2PrM + data.L2L*data.L2PrL

# Expected value difference (L-R)
data['vDiff'] = np.round(data.vL - data.vR,2)

# H's
data['H'] = np.maximum(data.L1H, data.L2H)

# Only keep the variables of interest
voi = ['subject','trial','rt','choice','vDiff','r','p','type','vL','vR','H']
data = data[voi]

# Ceiling of Maximum RT for PyDDM
maxRT = math.ceil(max(data.rt))

# Save & display
data.to_csv(datadir+"\\cleaned_data.csv", index=False)
data

,subject,trial,rt,choice,vDiff,r,p,type,vL,vR,H
0,9156374,1,5.968,0,-2.25,0.3,0.5,AB,30.00,32.25,75
1,9156374,2,3.920,1,3.75,0.3,0.5,AB,30.00,26.25,35
2,9156374,3,4.045,1,5.25,0.3,0.5,AB,30.00,24.75,25
3,9156374,4,4.630,1,-0.75,0.3,0.5,AB,29.25,30.00,55
4,9156374,5,5.052,1,0.00,0.3,0.5,AB,30.00,30.00,60
...,...,...,...,...,...,...,...,...,...,...,...
1003,9158535,36,3.026,0,3.75,0.3,0.5,ApBp,12.75,9.00,85
1004,9158535,37,1.884,0,-0.75,0.3,0.5,ApBp,8.25,9.00,55
1005,9158535,38,1.970,0,0.00,0.3,0.5,ApBp,9.00,9.00,60
1006,9158535,39,4.412,1,3.75,0.3,0.5,ApBp,9.00,5.25,35


---
# DDM: Condition Invariant

## Fit the average stimulus DDM

In [5]:
# Create a drift subclass so drift can vary with stimulus.
class DriftRate(ddm.models.Drift):
  name = "Drift depends linearly on value difference"
  required_parameters = ["driftrate"] # Parameters we want to include in the model.
  required_conditions = ["vDiff"] # The column in your sample data that modulates the parameters above.
  def get_drift(self, conditions, **kwargs):
    return self.driftrate * conditions["vDiff"]

# Define the model.
model_ci = Model(name="Standard DDM that does not distinguish between AB and A'B' choices",
                 drift=DriftRate(driftrate=Fittable(minval=-1, maxval=1.5)),
                 noise=NoiseConstant(noise=Fittable(minval=.001, maxval=2)),
                 bound=BoundConstant(B=1),
                 IC=ICPoint(x0=Fittable(minval=-.99, maxval=.99)),
                 overlay=OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.1)),
                 dx=.01, dt=.01, T_dur=maxRT,  # dx: spatial grid for evidence space (-B to B, in dx bins), dt: time step in s. See Shin et al 2022 Fig 4 for why I set dx=dt.
                 choice_names=("Left","Right"))

In [6]:
# Interactive plot! Play with the variables!
vDiff = np.sort(data.vDiff.unique())
pyddm.plot.model_gui_jupyter(model=model_ci, conditions={"vDiff":vDiff.tolist()})

Output()

In [7]:
# Only run this if specified at the start. Otherwise, just load pre-saved weights.
print("DDM: Condition Invariant.")

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:

    # Progress tracker.
    print("=========================")
    print("Subject " + str(subnum))

    # Subset the data.
    subdata = data[data["subject"]==subnum]

    # Create a sample object from our data. Sample objects are the standard input for pyDDM fitting functions.
    ddm_data = Sample.from_pandas_dataframe(subdata, rt_column_name="rt", choice_column_name="choice", choice_names=("Left","Right"))

    # Fit the model and show it off. Keep track of how long it took to estimate the parameters.
    clock = TicToc() # Timer
    clock.tic()
    set_N_cpus(ncpu) # Parallelize
    fit_model_ci = fit_adjust_model(sample=ddm_data, model=model_ci,
                                    fitting_method="differential_evolution",
                                    lossfunction=LossRobustBIC,
                                    verbose=False)
    clock.toc("Fitting subject " + str(subnum) + " took")
    display_model(fit_model_ci)

    # Save
    filename = ddmdir + "fit_model_ci_" + str(subnum) + ".txt"
    with open(filename, "w") as f:
      f.write(repr(fit_model_ci))

DDM: Condition Invariant.
Subject 9156374


Info: Params [0.04979574 0.36108265 0.01800779 0.09081022] gave 199.09996212323844


Fitting subject 9156374 took 95.149167 seconds.
Model Standard DDM that does not distinguish between AB and A'B' choices information:
Choices: 'Left' (upper boundary), 'Right' (lower boundary)
Drift component DriftRate:
    Drift depends linearly on value difference
    Fitted parameters:
    - driftrate: 0.049796
Noise component NoiseConstant:
    constant
    Fitted parameters:
    - noise: 0.361083
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.000000
IC component ICPoint:
    An arbitrary starting point.
    Fitted parameters:
    - x0: 0.018008
Overlay component OverlayNonDecision:
    Add a non-decision by shifting the histogram
    Fitted parameters:
    - nondectime: 0.090810
Fit information:
    Loss function: BIC
    Loss function value: 199.09996212323844
    Fitting method: differential_evolution
    Solver: auto
    Other properties:
        - nparams: 4
        - samplesize: 40
        - mess: ''

Subject 9156482


Info: Params [0.01783511 0.44195183 0.14867751 0.09391739] gave 209.2989920406309


Fitting subject 9156482 took 163.959048 seconds.
Model Standard DDM that does not distinguish between AB and A'B' choices information:
Choices: 'Left' (upper boundary), 'Right' (lower boundary)
Drift component DriftRate:
    Drift depends linearly on value difference
    Fitted parameters:
    - driftrate: 0.017835
Noise component NoiseConstant:
    constant
    Fitted parameters:
    - noise: 0.441952
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.000000
IC component ICPoint:
    An arbitrary starting point.
    Fitted parameters:
    - x0: 0.148678
Overlay component OverlayNonDecision:
    Add a non-decision by shifting the histogram
    Fitted parameters:
    - nondectime: 0.093917
Fit information:
    Loss function: BIC
    Loss function value: 209.2989920406309
    Fitting method: differential_evolution
    Solver: auto
    Other properties:
        - nparams: 4
        - samplesize: 40
        - mess: ''

Subject 9156521


...

Fitting subject 9158415 took 57.777790 seconds.
Model Standard DDM that does not distinguish between AB and A'B' choices information:
Choices: 'Left' (upper boundary), 'Right' (lower boundary)
Drift component DriftRate:
    Drift depends linearly on value difference
    Fitted parameters:
    - driftrate: 0.033980
Noise component NoiseConstant:
    constant
    Fitted parameters:
    - noise: 0.739526
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.000000
IC component ICPoint:
    An arbitrary starting point.
    Fitted parameters:
    - x0: -0.025123
Overlay component OverlayNonDecision:
    Add a non-decision by shifting the histogram
    Fitted parameters:
    - nondectime: 0.095536
Fit information:
    Loss function: BIC
    Loss function value: 139.7986041393538
    Fitting method: differential_evolution
    Solver: auto
    Other properties:
        - nparams: 4
        - samplesize: 40
        - mess: ''

Subject 9158418


Info: Params [0.0328282  0.54839512 0.02330064 0.09486222] gave 203.08192402182655


Fitting subject 9158418 took 60.411642 seconds.
Model Standard DDM that does not distinguish between AB and A'B' choices information:
Choices: 'Left' (upper boundary), 'Right' (lower boundary)
Drift component DriftRate:
    Drift depends linearly on value difference
    Fitted parameters:
    - driftrate: 0.032828
Noise component NoiseConstant:
    constant
    Fitted parameters:
    - noise: 0.548395
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.000000
IC component ICPoint:
    An arbitrary starting point.
    Fitted parameters:
    - x0: 0.023301
Overlay component OverlayNonDecision:
    Add a non-decision by shifting the histogram
    Fitted parameters:
    - nondectime: 0.094862
Fit information:
    Loss function: BIC
    Loss function value: 203.08192402182655
    Fitting method: differential_evolution
    Solver: auto
    Other properties:
        - nparams: 4
        - samplesize: 40
        - mess: ''

Subject 9158535


Info: Params [ 0.01255758  0.45559451 -0.03772968  0.04072278] gave 243.28623720943997


Fitting subject 9158535 took 42.144620 seconds.
Model Standard DDM that does not distinguish between AB and A'B' choices information:
Choices: 'Left' (upper boundary), 'Right' (lower boundary)
Drift component DriftRate:
    Drift depends linearly on value difference
    Fitted parameters:
    - driftrate: 0.012558
Noise component NoiseConstant:
    constant
    Fitted parameters:
    - noise: 0.455595
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.000000
IC component ICPoint:
    An arbitrary starting point.
    Fitted parameters:
    - x0: -0.037730
Overlay component OverlayNonDecision:
    Add a non-decision by shifting the histogram
    Fitted parameters:
    - nondectime: 0.040723
Fit information:
    Loss function: BIC
    Loss function value: 243.28623720943997
    Fitting method: differential_evolution
    Solver: auto
    Other properties:
        - nparams: 4
        - samplesize: 39
        - mess: ''



## Extract parameters and negative log likelihood for the model objects

In [8]:
# Placeholders
model_ci_bic = []
model_ci_drift = []
model_ci_noise = []
model_ci_bias = []
model_ci_ndt = []

# Iterate through subjects.
subnums = np.sort(data.subject.unique())
for subnum in subnums:
    
    # Load
    filename = ddmdir + "fit_model_ci_" + str(subnum) + ".txt"
    with open(filename, "r") as f:
        model_loaded = eval(f.read())

    # Negative Log Likelihood.
    model_ci_bic.append(model_loaded.get_fit_result().value())
    
    # Fitted parameters.
    model_ci_drift.append(model_loaded.parameters()['drift']['driftrate'])
    model_ci_noise.append(model_loaded.parameters()['noise']['noise'])
    model_ci_bias.append(model_loaded.parameters()['IC']['x0'])
    model_ci_ndt.append(model_loaded.parameters()['overlay']['nondectime'])
    
d = {'bic':model_ci_bic, "drift":model_ci_drift, "noise":model_ci_noise, "bias":model_ci_bias, "ndt":model_ci_ndt}
indiv_model_ci = pd.DataFrame(data=d)
indiv_model_ci

,bic,drift,noise,bias,ndt
0,199.099962,0.049796,0.361083,0.018008,0.090810
1,209.298992,0.017835,0.441952,0.148678,0.093917
2,252.659117,0.004355,0.446656,0.053172,0.094006
3,202.288530,0.029265,0.253189,-0.027938,0.092357
4,255.719944,0.005094,0.424681,0.129161,0.095220
5,199.518575,0.037935,0.531350,0.011992,0.093788
6,235.147851,0.019726,0.675092,-0.240983,0.004930
7,207.431084,0.016288,0.343640,0.139197,0.075921
8,227.521763,0.032238,0.391072,0.112686,0.097657
9,212.820188,0.009758,0.260492,-0.013377,0.096313


## Means of BIC and Estimates

Confidence intervals assume normal distribution.

In [9]:
summstats_model_ci = pd.DataFrame(data={"mean":indiv_model_ci.mean(), 
                                        "se":indiv_model_ci.sem(),
                                        "ci_lower":indiv_model_ci.mean()-1.96*indiv_model_ci.sem(),
                                        "ci_upper":indiv_model_ci.mean()+1.96*indiv_model_ci.sem()}).T
summstats_model_ci

,bic,drift,noise,bias,ndt
mean,209.165493,0.024239,0.421925,0.025817,0.086274
se,4.871806,0.002466,0.024474,0.018300,0.003928
ci_lower,199.616752,0.019406,0.373957,-0.010051,0.078574
ci_upper,218.714234,0.029072,0.469893,0.061686,0.093974


---
# DDM: Separately by Block (p,r)